In [44]:
import re
import string
import pickle
import pandas as pd
import numpy as np
from numpy.linalg import norm

from tqdm import tqdm

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [45]:
%run ../src/tree_spider.py
%run ../src/utils.py

In [11]:
glove_dict = load_glove_embeddings()

100%|███████████████████████| 400000/400000 [01:16<00:00, 5254.88it/s]


In [12]:
def flatten_table(t):
    if len(t) == 0:
        return []
    if isinstance(t[0], str):
        return t
    result = []
    for sub_lst in t:
        result += flatten_table(sub_lst)
    return result

def clean_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]', ' ', text)
    return text

In [26]:
def clean_title_for_table(title):
    title = title.lower().strip().replace('\t', ' ').replace('\n', ' ')
    title = title.replace('chapter', '').replace('article', '')
    title = re.sub('[^a-zA-Z]', ' ', title)
    return title.strip()

def gather_tables(ts):
    table_dict = ts.tables
    table_lst = []
    for title in table_dict.keys():
        for t_pair in table_dict[title]:
            t, href = t_pair
            t = np.array(t)
            if len(t.shape) == 2 and len(t) > 0:
                table_text = ' '.join(t.flatten())
                table_lst.append((clean_title_for_table(title), t, table_text, href))
    return table_lst

In [27]:
i = 0
ts = pickle.load(open('../data/scrapped/' + str(i) + '.pkl', 'rb'))

In [28]:
len(ts.tables.items())

185

In [31]:
# list(ts.tables.items())[:40]

In [32]:
table_lst = gather_tables(ts)
len(table_lst)

/var/folders/_1/6j5yb2d97mj8hmmsycj336j80000gn/T/ipykernel_90802/1976794182.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  t = np.array(t)


121

In [33]:
table_lst[0]

('fire department',
 array([['',
         'In this situation, the persons under 16 years of age should\nbe removed once other responders arrive or the condition in Subsection\nD1 above is met.']],
       dtype='<U147'),
 ' In this situation, the persons under 16 years of age should\nbe removed once other responders arrive or the condition in Subsection\nD1 above is met.',
 '/32099519')

In [38]:
table_tokens = ['agricultural', 'residential', 'commercial', 'business', 'industrial', 'district', 'housing']
zoning_tokens = ['zoning', 'zone']

In [42]:
def embedding(w, embed_dict):
    w = lemmatizer.lemmatize(w)
    try:
        return embed_dict[w]
    except:
        return None

def cos_similarity(v1, v2):
    cos_sim = np.dot(v1, v2)/(norm(v1)*norm(v2))
    return cos_sim
    
def relavance(text, topic_tokens):
    text_rep = []
    text_words = word_tokenize(text)
    for w in text_words:
        w_vec = embedding(w, glove_dict)
        if w_vec:
            text_rep.append(w_vec)
    if len(text_rep) == 0:
        return 0
    text_rep = np.array(text_rep).mean(axis = 0)
    
    topic_rep = []
    for w in topic_tokens:
        t_vec = embedding(w, glove_dict)
        if t_vec:
            topic_rep.append(t_vec)
    topic_rep = np.array(topic_rep).mean(axis = 0)
    
    return cos_similarity(text_rep, topic_rep)

def find_most_relevance_table(ts):
    table_lst = gather_tables(ts)
    print('=> # of tables found:', len(table_lst))
    
    sim_scores = {}
    for i in range(len(table_lst)):
        title, table, table_text, href = table_lst[i]
        table_text = clean_text(table_text)
        content_sim = relavance(table_text, table_tokens)
        title_sim = relavance(title, zoning_tokens)
        sim_scores[i] = (0.4 * content_sim + 0.6 * title_sim, title, table, table_text, href)
    
    top_tables = sorted(sim_scores.items(), key = lambda item: item[1][0], reverse = True)
    print('=> Top 3 tables in terms of similarity:')
    print()
    for t_idx in top_tables[:3]:
        print(table_lst[t_idx[0]])
        print()
        print()
    return top_tables

In [47]:
all_top_tables = []
for i in tqdm(range(100)):
    ts = pickle.load(open('../data/scrapped/' + str(i) + '.pkl', 'rb'))
    top_tables = find_most_relevance_table(ts)
    all_top_tables.append(top_tables)

  0%|                                         | 0/100 [00:00<?, ?it/s]/var/folders/_1/6j5yb2d97mj8hmmsycj336j80000gn/T/ipykernel_90802/1976794182.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  t = np.array(t)


=> # of tables found: 121


  1%|▎                                | 1/100 [00:00<00:24,  4.10it/s]

=> Top 3 tables in terms of similarity:

('annexing  vacating and re zoning', array([['Ord. No. 97-16', 'Alley in Block 1 of Norwood Addition'],
       ['Ord. No. 99-7', 'Alley in Block 22'],
       ['Ord. No. 99-8', 'Part of Lee St. South of Park Street'],
       ['Ord. No. 99-13',
        'Alley West of Lots 13 & 14 of Block 1 of Norwood Addition'],
       ['Ord. No. 2000-05', 'NE Corner of Lot 12 of Block 3']],
      dtype='<U57'), 'Ord. No. 97-16 Alley in Block 1 of Norwood Addition Ord. No. 99-7 Alley in Block 22 Ord. No. 99-8 Part of Lee St. South of Park Street Ord. No. 99-13 Alley West of Lots 13 & 14 of Block 1 of Norwood Addition Ord. No. 2000-05 NE Corner of Lot 12 of Block 3', '/32101151')


('annexing  vacating and re zoning', array([['Ord. No. 97-12', 'From R-1 to R-3',
        'Part of NE 1/4 of S 30, Twp 15 N, Range 32 W'],
       ['Ord. No. 97-14', 'From R-1 to R-3',
        'Part of NW 14/ of S 31, Twp 15 N, Range 32 W'],
       ['Ord. No. 98-5', 'From A-1 to R-1',
  

  2%|▋                                | 2/100 [00:00<00:24,  4.01it/s]

...

=> # of tables found: 180


 98%|███████████████████████████████▎| 98/100 [00:10<00:00,  5.81it/s]

=> Top 3 tables in terms of similarity:

('ii      establishment of districts  maps', array([['AR-1', 'Agricultural Residential District'],
       ['AR-2', 'Agricultural Residential District'],
       ['MR', 'Medium-Density Residential District'],
       ['GR', 'General Residential District'],
       ['HR-1', 'High-Density Residential District'],
       ['HR-2', 'High-Density Residential District'],
       ['UR', 'Urban Residential District'],
       ['RPC', 'Residential Planned Community District'],
       ['VRP', 'Vacation-Retirement-Residential-Park District']],
      dtype='<U45'), 'AR-1 Agricultural Residential District AR-2 Agricultural Residential District MR Medium-Density Residential District GR General Residential District HR-1 High-Density Residential District HR-2 High-Density Residential District UR Urban Residential District RPC Residential Planned Community District VRP Vacation-Retirement-Residential-Park District', '/8883884')


('ii      establishment of districts  ma

100%|███████████████████████████████| 100/100 [00:10<00:00,  9.18it/s]

=> Top 3 tables in terms of similarity:

('iii      zoning districts', array([['Land Use Category', 'Zoning District'],
       ['Residential Low', 'RS-70 Single Family Residential'],
       ['Residential Low', 'RS-50 Single Family Residential'],
       ['Residential Low', 'RS-35 Single Family Residential'],
       ['Residential Low Medium', 'RM-10 Residential Multifamily'],
       ['Residential Medium', 'RM-12.5 Residential Multifamily'],
       ['Residential Medium', 'RM-15 Residential Multifamily'],
       ['Residential Medium', 'MH-13.5 Mobile Home Residential'],
       ['Residential High', 'RM-15.5 Residential Multifamily'],
       ['Commercial General', 'Commercial General FAR 0.55'],
       ['Commercial water-dependent', 'Commercial General FAR 0.50'],
       ['Institutional', 'Commercial General FAR 0.65'],
       ['Institutional-public', 'Commercial General FAR 0.65'],
       ['Preservation', 'Aquatic Lands, Preservation'],
       ['Rec/Open Space',
        'All districts excep

In [ ]:
for i in tqdm(range(100)):
    ts = pickle.load(open('../data/scrapped/' + str(i) + '.pkl', 'rb'))

In [ ]:
# 1. filter out table with wrong format
# 2. save all the tables into some local format
# 3. change the scoring function